We are combining everything together. We will change the search term below and search for related subreddits (up to limit) and then parse 5 items in each and show thumbnail (if exists).

In [1]:
# Change these values
search_term = 'puppies'
sr_limit = 10
display_here = True
save_as_html = True
post_filter = 'top' # You can use 'top', 'hot', 'new', 'rising', 'random'
period = 'month' # You can use 'hour', 'day', 'week', 'month', 'year', 'all' (as a top filter)

In [2]:
# User-specific settings
import os
reddit_username = 'alpscode'
reddit_password = os.environ['reddit_pass']
app_id = 'QC-6FYIEH_kqUw'
app_secret = os.environ['reddit_secret']
reddit_header = {'user-agent': 'parser by alpscode'}

In [3]:
import requests
base_url = 'https://www.reddit.com/'
data = {'grant_type': 'password', 'username': reddit_username, 'password': reddit_password}
client_auth = requests.auth.HTTPBasicAuth(app_id, app_secret)
response = requests.post(base_url + 'api/v1/access_token',
                  data=data,
                  headers=reddit_header,
                  auth=client_auth)
values = response.json()
api_url = 'https://oauth.reddit.com'
token = 'bearer {}'.format(values['access_token'])
headers = {'Authorization': token, 'User-Agent': 'parser by {}'.format(reddit_username)}

In [4]:
payload = {'q': search_term, 'limit': sr_limit}
response = requests.get(api_url + '/subreddits/search', headers=headers, params=payload)
js = response.json()

sr = []
for i in range(js['data']['dist']):
    sr.append(js['data']['children'][i]['data']['display_name'])

print('Parsing following subreddits')
print(sr)

Parsing following subreddits
['puppies', 'aww', 'dogpictures', 'corgi', 'lookatmydog', 'pics', 'dogs', 'Eyebleach', 'goldenretrievers', 'husky']


In [5]:
payload = {'t': period, 'limit': 5}
imghtml = ''
for s in sr:
    imghtml += '<h3 style="clear:both">{}</h3><div>'.format(s)
    r = requests.get(api_url + '/r/{}/{}'.format(s, post_filter), headers=headers, params=payload)
    js = r.json()
    for i in range(js['data']['dist']):
        thumbnail = js['data']['children'][i]['data']['thumbnail']
        if thumbnail in ['', 'self', 'nsfw', 'spoiler', 'default']:
            continue
        imghtml += '<span style="float:left"><a href="{}"><img src="{}" title="{}" target="_blank" \></a></span>'.format(
            js['data']['children'][i]['data']['url'],
            thumbnail,
            js['data']['children'][i]['data']['title'],
        )
    imghtml += '</div>'

In [6]:
if display_here:
    from IPython.core.display import display, HTML
    display(HTML(imghtml))
if save_as_html:
    with open("{}.html".format(search_term), "w", encoding='utf-8') as html_page:
        html_page.write(imghtml)